<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/NGLextractionprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
NGL extraction process

In [2]:
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

# NGL extraction plant

In [10]:
#Feed gas
fluidcomposition = {'ComponentName':  ["nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane", "n-hexane", "C7", "C8", "C9", "C10-C11", "C12-C13", "C14-C15", "C16-C18", "C19-C20", "C21-C23", "C24-C29", "C30"],
      'MolarComposition[-]':  [0.5901, 0.00704, 80.423, 8.0634, 2.8112, 0.9095, 0.7896, 0.06577, 0.07712, 0.08545, 0.2025, 0.2507, 0.07571, 0.05015, 0.003053, 0.001353, 0.0004328, 0.00802, 0.000127, 0.0004921, 0.00009922],
      'MolarMass[kg/mol]': [None,None, None,None,None,None,None,None,None,None,0.09832, 0.11227, 0.12627, 0.14689, 0.174, 0.202, 0.237, 0.272, 0.307, 0.367, 0.594],
      'RelativeDensity[-]': [None,None, None,None,None,None,None,None,None,None, 0.737, 0.758, 0.775, 0.794, 0.814, 0.830, 0.846, 0.860, 0.872, 0.889, 0.935]
}
fluidcompositiondf = pd.DataFrame(fluidcomposition)
fluid1 = fluid_df(fluidcompositiondf, lastIsPlusFraction=False)
fluid1.setMixingRule('classic')
fluid1.setMultiPhaseCheck(True)

print('viscosity ', fluid1.getViscosity('cP'))


standardPressure = 1.01325
fluid1.setPressure(standardPressure, 'bara')
standardTemperature = 20.0 # C
fluid1.setTemperature(standardTemperature, 'C')

TPflash(fluid1)
fluid1.initProperties()
stdoildensity = fluid1.getPhase('oil').getDensity('kg/m3')
stdviscosity = fluid1.getPhase('oil').getViscosity('cP')
stdGOR = fluid1.getPhase('gas').getVolume('m3')/fluid1.getPhase('oil').getVolume('m3')
print('API oil ', (141.5/(stdoildensity/1e3)-131.5))
print('std viscosity oil ', stdviscosity, 'cP')
print('GOR ', stdGOR, ' Sm3 gas / Sm3 oil')

viscosity  0.0218781506556418
API oil  48.433703440199395
std viscosity oil  1.1357740274575026 cP
GOR  110226.37355245513  Sm3 gas / Sm3 oil


In [8]:
thermoOps = jNeqSim.thermodynamicOperations.ThermodynamicOperations(fluid1.clone())
thermoOps.calcPTphaseEnvelope()


#plt.plot(list(thermoOps.getOperation().get("dewT")),list(thermoOps.getOperation().get("dewP")), label="dew point")
#plt.plot(list(thermoOps.getOperation().get("bubT")),list(thermoOps.getOperation().get("bubP")), label="bubble point")
plt.plot(list(thermoOps.getOperation().get("dewT2")),list(thermoOps.getOperation().get("dewP2")), label="dew point")
plt.plot(list(thermoOps.getOperation().get("bubT2")),list(thermoOps.getOperation().get("bubP2")), label="bubble point")
plt.title('PT envelope')
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [bar]')
plt.legend()
plt.show()

java.lang.ArrayIndexOutOfBoundsException: ignored

In [11]:
#NGL expander code
from neqsim.process import stream, expander, heater, compsplitter, separator, heatExchanger
from neqsim.thermo import fluid, printFrame
from neqsim.process import compsplitter, compressor, splitter, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle2, setpoint, calculator

feedstream = stream(fluid1)

pressureNGL = 50.6
onshoreplant_inlet_pressure = 110.0
onshoreplant_inlet_temperature = 20.0

clearProcess()

inletheater = heater(feedstream)
inletheater.setOutPressure(onshoreplant_inlet_pressure, 'bara')
inletheater.setOutTemperature(onshoreplant_inlet_temperature, 'C')

heatEx = heatExchanger(inletheater.getOutStream())
heatEx.setName("cross heat-exchanger")
heatEx.setGuessOutTemperature(273.15 - 30.0)
heatEx.setUAvalue(3000e3) #assumes an UA vlue of 3000 MW/m2K

expanderKarsto = expander(heatEx.getOutStream(0), pressureNGL)
expanderKarsto.setOutletPressure(pressureNGL)
expanderKarsto.setIsentropicEfficiency(0.75) # assume same efficiency as for compressors

scrubberNGL = separator(expanderKarsto.getOutletStream())

recyOut = stream(scrubberNGL.getGasOutStream().clone())

gasResycle = recycle2("gas resycle")
gasResycle.addStream(scrubberNGL.getGasOutStream())
gasResycle.setOutletStream(recyOut)

heatEx.setFeedStream(1, recyOut)

gasfromdeethanizer = stream(heatEx.getOutStream(1).clone())

gasmixer = mixer('gas mixer')
gasmixer.addStream(heatEx.getOutStream(1))
gasmixer.addStream(gasfromdeethanizer)

dryGasExportCompressor = compressor(gasmixer.getOutletStream())
dryGasExportCompressor.setOutletPressure(100.0, 'bara')
dryGasExportCompressor.setIsentropicEfficiency(0.75)

exportDryGas = stream(dryGasExportCompressor.getOutStream())

NGLvalve1 = valve(scrubberNGL.getLiquidOutStream())
NGLvalve1.setOutletPressure(30.0)

#FRACTIONATION PROCESS
deethanizer = distillationColumn(trays=5, reboil=True, condenser=False, name="deethanizer")
deethanizer.addFeedStream(NGLvalve1.getOutStream(),5)
deethanizer.getReboiler().setOutTemperature(273.15+105.0)
deethanizer.setTopPressure(30.0)
deethanizer.setBottomPressure(32.0)

compressorgasdeethanizer = compressor(deethanizer.getGasOutStream())
compressorgasdeethanizer.setOutletPressure(pressureNGL, 'bara')

nglGasResycle = recycle2("gas resycle 2")
nglGasResycle.addStream(compressorgasdeethanizer.getOutStream())
nglGasResycle.setOutletStream(gasfromdeethanizer)

valvetodebutanizer = valve(deethanizer.getLiquidOutStream())
valvetodebutanizer.setOutletPressure(14.0)

debutanizer = distillationColumn(trays=10, reboil=True, condenser=True, name="debutanizer")
debutanizer.addFeedStream(valvetodebutanizer.getOutStream(),9)
debutanizer.getCondenser().setRefluxRatio(0.1)
debutanizer.getCondenser().setTotalCondenser(True)
debutanizer.getReboiler().setOutTemperature(273.15+163.0);
debutanizer.setTopPressure(12.8)
debutanizer.setBottomPressure(15.0)

onshoreprocess = getProcess()

In [ ]:
thread = onshoreprocess.runAsThread()
thread.join(60 * 60 * 1000)   # 60 min maks tid

In [ ]:
#mass balance onshore process should be close to 0% lost
massbalanceonshore = (feedstream.getOutStream().getFlowRate('kg/hr') - exportDryGas.getFlowRate('kg/hr') - debutanizer.getLiquidOutStream().getFlowRate('kg/hr') - debutanizer.getGasOutStream().getFlowRate('kg/hr') )/pipe1.getOutStream().getFlowRate('kg/hr')

print('mass lost % ',massbalanceonshore*100)

In [ ]:
print('feed gas')
printFrame(inletheater.getOutStream().getFluid())

In [ ]:
#De-ethanizer results
print('gas from deethanizer')
printFrame(deethanizer.getGasOutStream().getFluid())
print('liquid from deethanizer')
printFrame(deethanizer.getLiquidOutStream().getFluid())

In [ ]:
#debutanizer results
print('LPG from top debutanizer ')
printFrame(debutanizer.getGasOutStream().getFluid())
print('C5+ from bottom debutanizer ')
printFrame(debutanizer.getLiquidOutStream().getFluid())

In [ ]:
print('propane in dry export gas ', exportDryGas.getFluid().getComponent('propane').getx()*100.0, ' mol%')
print('LPG production rat ', debutanizer.getGasOutStream().getFlowRate('idSm3/hr')*24.0 , ' Sm3/day')
print('C5+ rate ', debutanizer.getLiquidOutStream().getFlowRate('idSm3/hr')*24.0, ' Sm3/day')
print('temperature in NGL scrubber ', expanderKarsto.getOutStream().getTemperature('C'))
print('temperature LPG extraction reflux condenser ', debutanizer.getGasOutStream().getTemperature('C'))
print('GCV ', exportDryGas.GCV()/1e6, ' MJ/m3')

print('duty deethanizer reboiler ', deethanizer.getReboiler().getDuty()/1e6, ' MW')
print('duty debutanizer reboiler ', debutanizer.getReboiler().getDuty()/1e6, ' MW')
print('duty debutanizer condenser ', debutanizer.getCondenser().getDuty()/1e6, ' MW')
propaneRecovery = 1.0 - exportDryGas.getFluid().getComponent('propane').getNumberOfmoles()/pipe1.getOutStream().getFluid().getComponent('propane').getNumberOfmoles()
ibutaneRecovery = 1.0 - exportDryGas.getFluid().getComponent('i-butane').getNumberOfmoles()/pipe1.getOutStream().getFluid().getComponent('i-butane').getNumberOfmoles()
nbutaneRecovery = 1.0 - exportDryGas.getFluid().getComponent('n-butane').getNumberOfmoles()/pipe1.getOutStream().getFluid().getComponent('n-butane').getNumberOfmoles()
print('propane recovery ' ,propaneRecovery)
print('i-butane recovery ' ,ibutaneRecovery)
print('n-butane recovery ' ,nbutaneRecovery)

In [ ]:
print('export gas')
printFrame(exportDryGas.getFluid())